In [1]:
from numpy.random import RandomState
import numpy as np
import qctoolkit as qtk

In [2]:
maia = {
    #'flags': '-l runtime=6:0:0',#membycore=4G',
    'ip': '131.152.25.38',
    'submission_script': '$HOME/bin/qsubqtk.sh',
    'username': 'changk',
    'compress': True,
}

qmsetting = {
    'program': 'gaussian',
    #'theory': 'pbe0',
    #'vdw': 'd3',
    'theory': 'ccsd(t)',
    'basis_set': 'aug-cc-pVTZ',
    #'openshell': 'restricted',
    'threads': 1,
    'save_density': True,
    'print_energy': True,
    #'charge_multiplicity': [0, 1]
}

covalentList = np.arange(0.5, 2.0, 0.08)
hbondList = np.arange(0.7, 4.0, 0.16)
HF_covalent = 0.93
HF_hbond = 1.88

In [3]:
def build(n=2, start=0, end=1000):
    
    rs = RandomState(n + start + end)
    size = end - start
    covalent = rs.randint(0, len(covalentList), size=(size, n))
    hbond = None
    if n > 1:
        hbond = rs.randint(0, len(hbondList), size=(size, n-1))
    
    mols = []
    for i in range(size):
        covalent_inds = covalent[i]
        hbond_inds = None
        if hbond is not None:
            hbond_inds = hbond[i]
            
        mol = qtk.Molecule()
        base_length = 0
        ZR = []
        for j in range(n):
            Hn = base_length
            Fn = base_length + covalentList[covalent_inds[j]]
            ZR.append([1, Hn, 0, 0])
            ZR.append([9, Fn, 0, 0])
            if n > 1 and j < n - 1:
                base_length += hbondList[hbond_inds[j]] + covalentList[covalent_inds[j]]
                
        ZR = np.array(ZR)
        
        ZR[:, 1] = ZR[:, 1] - covalentList[covalent_inds[0]]
        mol.build(ZR)
        mol.name = mol.name + '_%04d' % i
        mols.append(mol)
    return mols

def checkBondLength(mol):
    for i in range(mol.N - 1):
        print " %4.3f" % mol.distance(i, i+1),
    print "; %4.3f" % mol.distance(0, mol.N-1)
        
    

In [4]:
mols = build(2, 0, 5)
for i in range(len(mols)):
    print "molecule %2d:" % i
    checkBondLength(mols[i])
    print 

molecule  0:
 1.700  1.660  0.820 ; 4.180

molecule  1:
 0.740  1.340  1.060 ; 3.140

molecule  2:
 1.620  3.260  1.140 ; 6.020

molecule  3:
 1.620  1.820  1.300 ; 4.740

molecule  4:
 1.140  2.620  1.060 ; 4.820



In [5]:
mols_3 = build(3)
inps_3 = [qtk.QMInp(m, **qmsetting) for m in mols_3]
mols_4 = build(4)
inps_4 = [qtk.QMInp(m, **qmsetting) for m in mols_4]
mols_5 = build(5)
inps_5 = [qtk.QMInp(m, **qmsetting) for m in mols_5]
mols_6 = build(6)
qmsetting['threads'] = 8
inps_6 = [qtk.QMInp(m, **qmsetting) for m in mols_6]
inps_3[10].write()
inps_4[20].write()
inps_5[30].write()
inps_6[40].write()

%nproc=1
%chk=tmp.chk
# ccsd(t)/aug-cc-pVTZ force int(grid=ultrafine) 6d 10f nosymm ExtraLinks=L608 Scf(maxcycle=1000,verytight) IOp(2/12=3)

H3F3_0010

0   1
H   -1.1400   0.0000   0.0000
F    0.0000   0.0000   0.0000
H    1.9800   0.0000   0.0000
F    2.6400   0.0000   0.0000
H    6.2200   0.0000   0.0000
F    7.8400   0.0000   0.0000


%nproc=1
%chk=tmp.chk
# ccsd(t)/aug-cc-pVTZ force int(grid=ultrafine) 6d 10f nosymm ExtraLinks=L608 Scf(maxcycle=1000,verytight) IOp(2/12=3)

H4F4_0020

0   1
H   -0.8200   0.0000   0.0000
F    0.0000   0.0000   0.0000
H    3.5800   0.0000   0.0000
F    5.0400   0.0000   0.0000
H    6.8600   0.0000   0.0000
F    8.4800   0.0000   0.0000
H    9.3400   0.0000   0.0000
F   10.6400   0.0000   0.0000


%nproc=1
%chk=tmp.chk
# ccsd(t)/aug-cc-pVTZ force int(grid=ultrafine) 6d 10f nosymm ExtraLinks=L608 Scf(maxcycle=1000,verytight) IOp(2/12=3)

H5F5_0030

0   1
H   -1.8600   0.0000   0.0000
F    0.0000   0.0000   0.0000
H    2.9400   0.0000   0.0000
F    4.72

In [6]:
#qtk.submit(inps_3, 'wavelet_m3_ccsdt', **maia)
#qtk.submit(inps_4, 'wavelet_m4_ccsdt', **maia)
#maia['flags'] = '-l runtime=168:0:0'#,#membycore=4G',
#qtk.submit(inps_5, 'wavelet_m5_ccsdt', **maia)
#qtk.submit(inps_6, 'wavelet_m6_ccsdt', **maia)

# qtk.qmRunAll(inps_3, 'wavelet_m3_local')